In [ ]:
import bs4
import requests
import re

In [230]:
url="https://www.edc.dk/sog/?ejd-typer=1&antal=1000&side=1#lstsort"
retVal=requests.get(url)

In [ ]:
retVal.status_code

In [232]:
htmlText=retVal.text

In [234]:
soup=bs4.BeautifulSoup(htmlText)

In [235]:
elements=soup.find_all("div", class_="row propertyitem__body--view")

Investigate one element

In [225]:
price=elements[0].find_all("div", class_="propertyitem__price")

In [227]:
rPris=re.sub("[^0-9]","",price[0].text)

In [38]:
tableInfo=elements[0].find_all("td")

In [141]:
retVal=elements[0].find_all("h2", class_="propertyitem__address--listview")

In [189]:
addrItems=retVal[0].find_all("span")

In [192]:
street=addrItems[0].text

In [194]:
azip=addrItems[1].text

In [200]:
import pymongo

In [201]:
client=pymongo.MongoClient("mongodb://localhost:27017")

In [202]:
db=client.homes

In [ ]:
for element in elements:
    dbDict={}
    #extracting price
    price=element.find_all("div", class_="propertyitem__price")
    rPris=int(re.sub("[^0-9]","",price[0].text))
    dbDict['pris']=rPris

    #extracting tableinfo
    tableInfo=element.find_all("td")
    if tableInfo[0].text.isnumeric():
        
        #get location
        retVal=element.find_all("h2", class_="propertyitem__address--listview")
        addrItems=retVal[0].find_all("span")
        street=addrItems[0].text
        mzip=addrItems[1].text
        dbDict['address']={"street":street,"zip":mzip}
        
        tempSid=element.find("meta")
        ttempSid=tempSid['content']
        retVal=re.search("=([0-9]+)$",ttempSid)
        sid=retVal.groups()[0]
        
        dbDict['_id']=sid
        dbDict['kvm']=int(tableInfo[0].text)
        dbDict['grund']=int(re.sub("[^0-9]","",tableInfo[1].text))
        dbDict['rum']=int(tableInfo[2].text)
        dbDict['buildYear']=int(tableInfo[3].text)
        dbDict['liggetid']=int(re.sub("[^0-9]","",tableInfo[4].text))
        dbDict['kvmpris']=int(re.sub("[^0-9]","",tableInfo[6].text))
        try:
            dbDict['ejerudg']=int(re.sub("[^0-9]","",tableInfo[7].text))
        except:
            print("Ups")
            dbDict['ejerudg']="Not found"
        print(dbDict)
        #re.sub("[^0-9]","",price[0].text)
        try:
            insret=db.homes.insert_one(dbDict)
            print(insret.inserted_id)
        except:
            print("doublet .. continue")
    else:
        print("No sqm. Skipping")